In [7]:
import boto3
session = boto3.session.Session(profile_name='prod', region_name='eu-west-1')
s3 = session.client('s3')
glue = session.client('glue')

In [ ]:
Bucket = 'sds-dev-ingestion-ingest-external-yield-management-files'
Prefix = 'processed/yi_fct_steerings_history'
paginator = s3.get_paginator('list_objects_v2')
operation_parameters = {'Bucket': Bucket,
                        'Prefix': Prefix}
last_files = []
for page in paginator.paginate(**operation_parameters):
    last_files.append(sorted(page['Contents'], key=lambda v: v['LastModified'], reverse=True)[0])    
last_file = sorted(last_files, key=lambda v: v['LastModified'], reverse=True)[0]
print(f"Last file: {last_file}")

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
table = pq.read_table(f"s3://{Bucket}/{last_file['Key']}")
print(table.schema)

In [8]:
paginator = glue.get_paginator('get_job_runs')
operation_parameters = {'JobName': 'sds-prod-csv-to-parquet'}
for page in paginator.paginate(**operation_parameters):
    print(page)
    break


{'JobRuns': [{'Id': 'jr_eb5e6bf81034c36da9a2d52d9ebb7029077d95a577957f2a03e3cb985949411e', 'Attempt': 0, 'JobName': 'sds-prod-csv-to-parquet', 'StartedOn': datetime.datetime(2024, 4, 19, 14, 53, 23, 114000, tzinfo=tzlocal()), 'LastModifiedOn': datetime.datetime(2024, 4, 19, 14, 54, 13, 642000, tzinfo=tzlocal()), 'CompletedOn': datetime.datetime(2024, 4, 19, 14, 54, 13, 642000, tzinfo=tzlocal()), 'JobRunState': 'SUCCEEDED', 'Arguments': {'--s3_file': 'sds-prod-ingest-external-sftp-files/yield-sftp-user/CompetitorPrice/SIXT-GDS-US-US-P-Friday-20240419-SubmitTime-1243-0000N_51169_2.csv', '--s3_output_file_location': 'sds-prod-ingest-external-sftp-files-out', '--s3_output_file_key': 'yield-sftp-user/CompetitorPrice/year=2024/month=04/day=19/SIXT-GDS-US-US-P-Friday-20240419-SubmitTime-1243-0000N_51169_2-1713527594673.parquet', '--config': 'sds-prod-ingest-configs/sftp/yield-sftp-user/CompetitorPrice/config.json', '--encryption': 'False'}, 'PredecessorRuns': [], 'AllocatedCapacity': 4, 'Exec